## シミュレーションを流し、学習するファイル

In [23]:
#path
using CUDA
using EzXML
using MDToolbox
device!(1)
pdb_filepath = "./alanine-dipeptide-nowater.pdb" #pdbのパス
ff_start_filepath  = "./data/amber14/protein.ff14SB.xml" #startのffのパス
out_dir = "sim_result" #出力するディレクトリ
isdir(out_dir) || mkdir(out_dir)

#setting
username = "ysako" #自分のusername
njobs = 10 #並列して流すシミュレーショ"ンの数
nepoch = 50
#nsteps = 10000000 #シミュレーションのステップ数
nsteps = 1000000

gpu_id = "n5"
update_ratio = 0.20 #forcefieldをどの程度変えるか

0.2

## シミュレーション

In [8]:
# 一つのシミュレーションを流すジョブスクリプト
# ff_filepath　　　  :xml formatの力場ファイルのパス
# out_traj_dir      :アウトプットするディレクトリのパス
# i                 :流すシミュレーションの通し番号

function run_job(ff_filepath, traj_filepath, log_filepath, i)
    sbatch_file = "temp_epoch_$(i).sh"
    
    open(sbatch_file, "w") do of
        println(of, "#!/bin/bash")
        println(of, "#SBATCH -p all")
        println(of, "#SBATCH -J sim$(i) # job name")
        println(of, "#SBATCH -n 1  # num of total mpi processes")
        println(of, "#SBATCH -c 1  # num of threads per mpi processes")
        println(of, "#SBATCH -o $(log_filepath)")
        println(of, "python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
    end
    sleep(5)
    run(`sbatch --gpus-per-node=1 -w $(gpu_id) $(sbatch_file)`)
    sleep(5)
    rm(sbatch_file)
end

run_job (generic function with 1 method)

## ffを作成する関数

In [10]:
phi_atom_type = ["protein-C", "protein-N", "protein-CX", "protein-C"]
psi_atom_type = ["protein-N", "protein-CX", "protein-C", "protein-N"]
function create_ff(ff_input_filepath, ff_output_filepath, update_ratio)
    input_xml = readxml(ff_input_filepath)
    output_xml = deepcopy(input_xml)
    xmlroot = root(output_xml)
    children = elements(xmlroot)
    children_name = nodename.(children)
    torsion_index = children_name .== "PeriodicTorsionForce"
    torsion = children[torsion_index][1] #これでtorsion内の要素にアクセスできる

    for ff_params in eachelement(torsion)
        params_name = [nodename(i) for i in eachattribute(ff_params)]
        params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        atom_type = [params_content[params_name .== "type$(i)"][1] for i in 1:4]
        if atom_type == phi_atom_type
            for a in eachattribute(ff_params)
                if occursin(r"k.*", a.name)
                    #ここにkを変える処理を入力
                    k = parse(Float32, a.content)
                    k_updated = k * (1 + update_ratio*rand())
                    a.content = string(k_updated)
                end
            end
            params_name = [nodename(i) for i in eachattribute(ff_params)]
            params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        end
        #ここから処理を書き忘れていたため追加した部分
        if atom_type == psi_atom_type
            for a in eachattribute(ff_params)
                if occursin(r"k.*", a.name)
                    #ここにkを変える処理を入力
                    k = parse(Float32, a.content)
                    k_updated = k * (1 + update_ratio*rand())
                    a.content = string(k_updated)
                end
            end
            params_name = [nodename(i) for i in eachattribute(ff_params)]
            params_content = [nodecontent(i) for i in eachattribute(ff_params)]
        end
        #ここまで
    end
    write(ff_output_filepath, output_xml)
end

create_ff(ff_start_filepath, "./new.xml", 0.001)

224141

In [ ]:
##最初のtrajを生成
for i in 1:njobs
    epoch_dir = joinpath(out_dir, "epoch$(i)") #epoch(i)のディレクトリ
    isdir(epoch_dir) || mkdir(epoch_dir)

    ff_filepath = joinpath(epoch_dir, "sim_$(i).xml")
    traj_filepath = joinpath(epoch_dir, "traj_$(i)")
    log_filepath = joinpath(epoch_dir, "sim_$(i).log")
    #create_ff_qoulomb(ff_start_filepath, ff_filepath, update_ratio)
    create_ff(ff_start_filepath, ff_filepath, update_ratio)

    run_job(ff_filepath, traj_filepath,log_filepath, i)
end

#シミュレーションが全て終わるまで待機
completed = false
while(!completed)
    n_completed = njobs - length(readlines(`squeue -u $(username)`)) + 1 #終了したシミュレーションの数
    #シミュレーションが終わっていないとき
    if(n_completed < njobs)
        println("--- $(n_completed) of $(njobs) jobs completed")
        sleep(600) #10分待機
    #全てのシミュレーションが終わったとき
    else
        completed = true
    end
end
println("--- all jobs completed")

Submitted batch job 1019
Submitted batch job 1020
Submitted batch job 1021
Submitted batch job 1022
Submitted batch job 1023
Submitted batch job 1024
Submitted batch job 1025
Submitted batch job 1026
Submitted batch job 1027
Submitted batch job 1028
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
--- 0 of 10 jobs completed
